In [5]:
import requests
from bs4 import BeautifulSoup
import sqlite3


def setup_database():
    conn = sqlite3.connect("countries.db")
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS countries (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            capital TEXT,
            population INTEGER,
            area REAL
        )
    ''')
    conn.commit()
    conn.close()

def scrape_countries():
    url = "https://www.scrapethissite.com/pages/simple/"
    response = requests.get(url)
    if response.status_code != 200:
        print("Failed to fetch the webpage.")
        return []

    soup = BeautifulSoup(response.content, "html.parser")
    countries = []
    country_divs = soup.find_all("div", class_="country")[:20]  # فقط 20 کشور اول
    for country_div in country_divs:
        name = country_div.find("h3", class_="country-name").text.strip()
        capital = country_div.find("span", class_="country-capital").text.strip()
        population = int(country_div.find("span", class_="country-population").text.strip().replace(",", ""))
        area = float(country_div.find("span", class_="country-area").text.strip().replace(",", ""))
        countries.append((name, capital, population, area))

    return countries

def save_to_database(countries):
    conn = sqlite3.connect("countries.db")
    cursor = conn.cursor()
    for country in countries:
        cursor.execute('''
            INSERT INTO countries (name, capital, population, area)
            VALUES (?, ?, ?, ?)
        ''', country)
    conn.commit()
    conn.close()

if __name__ == "__main__":
    setup_database()
    countries = scrape_countries()
    if countries:
        save_to_database(countries)
        print("Data has been successfully saved to the database.")
    else:
        print("No data was scraped.")


Data has been successfully saved to the database.
